In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables and initialize OpenAI client
# Make sure you have a .env file with your OPENAI_API_KEY
load_dotenv()
client = OpenAI(
    base_url = "https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY")
)

# --- Helper Function for API Calls ---
def call_openai(system_prompt, user_prompt, model="gpt-4o"):
    """Simple wrapper for OpenAI API calls."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {e}"


# --- Agent Class Implementations ---

class CodeGenerationAgent:
    """Agent that generates code based on a natural language description."""
    def __init__(self):
        self.description = "Code Generation Agent: Use this agent to generate code in various programming languages based on a textual description of the desired functionality."
        self.name = "Code Generation Agent"

    def get_description(self):
        """Returns the agent's description."""
        return self.description

    def run(self, task_prompt):
        """Executes the code generation task."""
        system_prompt = """You are a specialist Code Generation Agent. Your task is to write clean, efficient, and well-documented code based on the user's request.
        Provide only the code block for the requested language, followed by a brief explanation of how it works."""
        
        user_prompt = f"Generate code for the following task: {task_prompt}"
        return call_openai(system_prompt, user_prompt)


class EuropeanHistoryQAAgent:
    """Agent that answers questions about European history."""
    def __init__(self):
        self.description = "European History Q&A Agent: Use this agent to get answers to specific questions about European history, from ancient times to the modern era."
        self.name = "European History Q&A Agent"

    def get_description(self):
        """Returns the agent's description."""
        return self.description

    def run(self, task_prompt):
        """Executes the history question-answering task."""
        system_prompt = """You are a specialist European History Agent. Your task is to provide accurate and detailed answers to questions about European history.
        Cite key dates, figures, and events in your response."""
        
        user_prompt = f"Answer the following European history question: {task_prompt}"
        return call_openai(system_prompt, user_prompt)


class MathematicalProblemSolvingAgent:
    """Agent that solves mathematical problems."""
    def __init__(self):
        self.description = "Mathematical Problem Solving Agent: Use this agent to solve mathematical problems, including algebra, calculus, and other quantitative tasks."
        self.name = "Mathematical Problem Solving Agent"

    def get_description(self):
        """Returns the agent's description."""
        return self.description

    def run(self, task_prompt):
        """Executes the mathematical problem-solving task."""
        system_prompt = """You are a specialist Mathematical Problem Solving Agent. Your task is to solve the given mathematical problem, showing the steps involved for clarity.
    Provide the final answer clearly."""
        
        user_prompt = f"Solve the following mathematical problem: {task_prompt}"
        return call_openai(system_prompt, user_prompt)


# --- Routing Agent with Dynamic Prompt Generation ---
def routing_agent(task_prompt, agents):
    """
    Routing agent that uses an LLM to determine which agent to use based on the task prompt.
    
    Args:
        task_prompt (str): The user's request.
        agents (list): A list of agent objects to choose from.
    """
    
    # Dynamically create the list of available agents from their descriptions
    agent_descriptions = "\n".join([f"- {agent.get_description()}" for agent in agents])
    
    system_prompt = f"""You are an expert AI routing assistant. Your job is to analyze a user's task prompt and route it to the most appropriate agent.
    You have the following agents available:
    {agent_descriptions}

    Analyze the user's task prompt below and determine which agent is the best fit.
    Respond only with the exact name of the agent (e.g., 'Code Generation Agent'), and nothing else.

    Task: {task_prompt}"""
    
    user_prompt = f"Given the task: '{task_prompt}', which agent should handle this task?"
    
    # Use an LLM to choose the agent
    agent_choice_name = call_openai(system_prompt, user_prompt).strip()
    
    # Find the chosen agent object and run it
    for agent in agents:
        if agent.name == agent_choice_name:
            print(f"--- Routing task to {agent.name}... ---")
            return agent.run(task_prompt)
            
    return f"Error: Could not find an agent named '{agent_choice_name}'. Please check the routing prompt."


# --- Example Usage ---
if __name__ == "__main__":
    # 1. Initialize all available agents
    code_agent = CodeGenerationAgent()
    history_agent = EuropeanHistoryQAAgent()
    math_agent = MathematicalProblemSolvingAgent()
    
    all_agents = [code_agent, history_agent, math_agent]

    # --- Example 1: Code Generation Task ---
    task1_prompt = "Write a Python function that implements a recursive binary search algorithm."
    print(f"User Task: \"{task1_prompt}\"")
    result1 = routing_agent(task1_prompt, all_agents)
    print("\nAgent Output:\n", result1)
    print("\n" + "="*50 + "\n")

    # --- Example 2: European History Task ---
    task2_prompt = "What were the main causes of the French Revolution?"
    print(f"User Task: \"{task2_prompt}\"")
    result2 = routing_agent(task2_prompt, all_agents)
    print("\nAgent Output:\n", result2)
    print("\n" + "="*50 + "\n")

    # --- Example 3: Mathematical Task ---
    task3_prompt = "Calculate the derivative of f(x) = 5x^4 + 3x^2 - 2x + 7."
    print(f"User Task: \"{task3_prompt}\"")
    result3 = routing_agent(task3_prompt, all_agents)
    print("\nAgent Output:\n", result3)

User Task: "Write a Python function that implements a recursive binary search algorithm."
--- Routing task to Code Generation Agent... ---

Agent Output:
 ```python
def binary_search_recursive(arr, target, low, high):
    """
    Perform a binary search on a sorted array using recursion.

    Parameters:
    arr (list): The sorted list to search.
    target (int): The element to search for.
    low (int): The lower index of the sublist to search.
    high (int): The upper index of the sublist to search.

    Returns:
    int: The index of the target element if found, otherwise -1.
    """
    if low > high:
        return -1  # Base case: target is not found

    mid = (low + high) // 2  # Calculate the middle index

    if arr[mid] == target:
        return mid  # Target found at mid index
    elif arr[mid] > target:
        return binary_search_recursive(arr, target, low, mid - 1)  # Search in the left half
    else:
        return binary_search_recursive(arr, target, mid + 1, high) 

In [5]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables and initialize OpenAI client
load_dotenv()
client = OpenAI(
    base_url = "https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY")
)

# --- Helper Function for API Calls ---
def call_openai(system_prompt, user_prompt, model="gpt-3.5-turbo"):
    """Simple wrapper for OpenAI API calls."""
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    return response.choices[0].message.content


# --- Agents for Different Retail Tasks ---

def product_researcher_agent(query):
    system_prompt = """You are a product research agent for a retail company. Your task is to provide 
    structured information about products, market trends, and competitor pricing."""
    
    user_prompt = f"Research this product thoroughly: {query}"
    return call_openai(system_prompt, user_prompt)


def customer_analyzer_agent(query):
    system_prompt = """You are a customer analysis agent. Your task is to analyze customer feedback, 
    preferences, and purchasing patterns."""
    
    user_prompt = f"Analyze customer behavior for: {query}"
    return call_openai(system_prompt, user_prompt)


def pricing_strategist_agent(query, product_data=None, customer_data=None):
    system_prompt = """You are a pricing strategist agent. Your task is to recommend optimal pricing 
    strategies based on product research and customer analysis. You are given both sets of data and must return a clear recommendation."""
    
    user_prompt = f"""We are launching: {query}
    
Product Research Results:
{product_data or 'Not provided'}

Customer Analysis Results:
{customer_data or 'Not provided'}

Based on this information, what should be the optimal pricing strategy?
"""
    return call_openai(system_prompt, user_prompt)


# --- Routing Agent with LLM-Based Task Determination ---
def routing_agent(query, *args):
    """Determines the task type and routes to the appropriate agent(s)."""
    
    system_prompt = """You are a task classification agent. Based on the user's query, classify it into one of:
- "product research"
- "customer analysis"
- "pricing strategy"

Respond with only the category name, no punctuation or explanation."""
    
    task_type = call_openai(system_prompt, query).strip().lower()

    if task_type == "product research":
        return product_researcher_agent(query)
    
    elif task_type == "customer analysis":
        return customer_analyzer_agent(query)
    
    elif task_type == "pricing strategy":
        # Use passed args if available
        product_data = args[0] if len(args) > 0 else product_researcher_agent(query)
        customer_data = args[1] if len(args) > 1 else customer_analyzer_agent(query)
        return pricing_strategist_agent(query, product_data, customer_data)
    
    else:
        return f"❌ Unknown task type: {task_type}"


# --- Example Usage ---
if __name__ == "__main__":
    queries = [
        "What are the specifications and current market trends for wireless earbuds?",
        "What do customers think about our premium coffee brand?",
        "What should be the optimal price for our new organic skincare line?"
    ]
    
    for query in queries:
        print(f"\n=== Query: {query} ===")
        print("🔄 Processing...\n")
        result = routing_agent(query)
        print("✅ Result:\n")
        print(result)
        print("\n" + "="*60)


=== Query: What are the specifications and current market trends for wireless earbuds? ===
🔄 Processing...

✅ Result:

**Specifications for Wireless Earbuds:**

1. **Wireless Technology:** Most wireless earbuds use Bluetooth technology to connect to devices.
2. **Battery Life:** Typically ranges from 4 to 12 hours on a single charge, with the charging case providing additional charges.
3. **Sound Quality:** High-quality wireless earbuds offer clear sound with good bass and noise cancellation features.
4. **Fit and Comfort:** Different earbud designs (in-ear, over-ear, etc.) cater to various preferences for fit and comfort.
5. **Water Resistance:** Some earbuds are water-resistant or waterproof, suitable for sports and outdoor activities.
6. **Controls:** Touch controls or physical buttons on the earbuds allow users to adjust volume, change tracks, answer calls, etc.
7. **Microphone:** Built-in microphones enable hands-free calling and voice assistant access.
8. **Compatibility:** Wire